In [1]:
# Import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout
from keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.wrappers.scikit_learn import KerasRegressor
from math import floor
from sklearn.metrics import make_scorer, accuracy_score
from bayes_opt import BayesianOptimization
from sklearn.model_selection import StratifiedKFold
from keras.layers import LeakyReLU
LeakyReLU = LeakyReLU(alpha=0.1)
import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)


df = pd.read_csv('../data/data2.csv')
df.info()

2022-09-12 23:28:01.128087: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2112 entries, 0 to 2111
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ID         2112 non-null   int64  
 1   Substrate  2112 non-null   object 
 2   Ceramic    2112 non-null   object 
 3   Thickness  2112 non-null   float64
 4   Lsub       2112 non-null   float64
 5   asub       2112 non-null   float64
 6   bsub       2112 non-null   float64
 7   Lcer       2112 non-null   float64
 8   acer       2112 non-null   float64
 9   bcer       2112 non-null   float64
 10  L          2112 non-null   float64
 11  a          2112 non-null   float64
 12  b          2112 non-null   float64
dtypes: float64(10), int64(1), object(2)
memory usage: 214.6+ KB


In [8]:
# Use scikit-learn to grid search the batch size and epochs
import numpy as np
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor

# split into input (X) and output (y) variables
X, y = df[['Thickness','Lsub', 'asub','bsub', 'Lcer', 'acer', 'bcer']].to_numpy(), df[['L', 'a', 'b']].to_numpy()

# define the keras model
model = Sequential()
model.add(Dense(7, input_dim=7, activation='relu'))
model.add(Dense(3, activation='linear'))
# compile the keras model
model.compile(loss='mae', optimizer='adam', metrics=['accuracy'])
# fit the keras model on the dataset
model.fit(X, y, epochs=150, batch_size=10)
# evaluate the keras model
_, accuracy = model.evaluate(X, y)
print('Accuracy: %.2f' % (accuracy*100))

Epoch 1/150


2022-09-12 23:38:10.982496: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


212/212 [==============================] - 0s 679us/step - loss: 40.9777 - accuracy: 0.0000e+00
Epoch 2/150
212/212 [==============================] - 0s 642us/step - loss: 14.4486 - accuracy: 0.7325
Epoch 3/150
212/212 [==============================] - 0s 637us/step - loss: 4.4810 - accuracy: 1.0000
Epoch 4/150
212/212 [==============================] - 0s 631us/step - loss: 3.4581 - accuracy: 1.0000
Epoch 5/150
212/212 [==============================] - 0s 641us/step - loss: 2.8361 - accuracy: 1.0000
Epoch 6/150
212/212 [==============================] - 0s 638us/step - loss: 2.4670 - accuracy: 1.0000
Epoch 7/150
212/212 [==============================] - 0s 643us/step - loss: 2.2698 - accuracy: 1.0000
Epoch 8/150
212/212 [==============================] - 0s 699us/step - loss: 2.1206 - accuracy: 1.0000
Epoch 9/150
212/212 [==============================] - 0s 682us/step - loss: 2.0253 - accuracy: 1.0000
Epoch 10/150
212/212 [==============================] - 0s 727us/step - loss: 1

Epoch 80/150
212/212 [==============================] - 0s 625us/step - loss: 1.8468 - accuracy: 1.0000
Epoch 81/150
212/212 [==============================] - 0s 626us/step - loss: 1.8511 - accuracy: 1.0000
Epoch 82/150
212/212 [==============================] - 0s 625us/step - loss: 1.8486 - accuracy: 1.0000
Epoch 83/150
212/212 [==============================] - 0s 648us/step - loss: 1.8544 - accuracy: 1.0000
Epoch 84/150
212/212 [==============================] - 0s 658us/step - loss: 1.8453 - accuracy: 1.0000
Epoch 85/150
212/212 [==============================] - 0s 638us/step - loss: 1.8503 - accuracy: 1.0000
Epoch 86/150
212/212 [==============================] - 0s 651us/step - loss: 1.8417 - accuracy: 1.0000
Epoch 87/150
212/212 [==============================] - 0s 685us/step - loss: 1.8496 - accuracy: 1.0000
Epoch 88/150
212/212 [==============================] - 0s 700us/step - loss: 1.8467 - accuracy: 1.0000
Epoch 89/150
212/212 [==============================] - 0s 696us

In [12]:

def create_model():
    model = Sequential()
    model.add(Dense(7, input_dim=7, activation='relu'))
    model.add(Dense(3, activation='linear'))
    # compile the keras model
    model.compile(loss='mae', optimizer='adam', metrics=['accuracy'])
    

# create model
model = KerasRegressor(model=create_model)
# define the grid search parameters
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

AttributeError: 'KerasRegressor' object has no attribute '__call__'